# Examples

## Query Example

This notebook will cover example usage of `Optimization` record. As a note we will be using the MolSSI QCArchive server as a data source. Any `id`s used in this example will not be valid for local servers.

In [1]:
import qcfractal.interface as ptl
client = ptl.FractalClient()
client

FractalClient(server_name='The MolSSI QCArchive Server', address='https://api.qcarchive.molssi.org:443/', username='None')

We can query results from the database based off a variety of values, but for this example we will query a known result from the database.

In [2]:
record = client.query_procedures(id=1683293)[0]
record

<OptimizationRecord(id='1683293' status='COMPLETE')>

There are a variety of helper functions on this object to find quantities related to the computation.

In [3]:
record.get_final_molecule()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Molecule(name='C14H14O' formula='C14H14O' hash='2594158')>

In [4]:
record.show_history()

We can also observe the program, method, and basis for which the optimization was executed under.

In [5]:
record.qc_spec.dict()

{'driver': <DriverEnum.gradient: 'gradient'>,
 'method': 'b3lyp-d3(bj)',
 'basis': 'dzvp',
 'keywords': None,
 'program': 'psi4'}

We can also find all keywords passed into the geometry optimization. Here we see that this geometry optimization was evaluated under a dihedral constraint.

In [6]:
record.keywords

{'coordsys': 'tric',
 'enforce': 0.1,
 'reset': True,
 'qccnv': True,
 'epsilon': 0,
 'constraints': {'set': [{'type': 'dihedral',
    'indices': [9, 10, 13, 14],
    'value': 30}]},
 'program': 'psi4'}

Finally, every Result generated in the computational trajectory can be queried and observed. Here we will obtain the very last computed Result.

In [7]:
record.get_trajectory()[-1]

<ResultRecord(id='1467860' status='COMPLETE')>

## Compute Example

Computation of a result can be accomplished by specifying all parameters to a quantum chemistry computation and a molecule. An example can be seen below using a FractalSnowflake instance.

In [8]:
from qcfractal import FractalSnowflakeHandler
snowflake = FractalSnowflakeHandler()
client = snowflake.client()
client

FractalClient(server_name='FractalSnowFlake_db_f1c89', address='https://localhost:57605/', username='None')

In [9]:
methane = ptl.Molecule.from_data("pubchem:methane")
methane

	Searching PubChem database for methane (single best match returned)
	Found 1 result(s)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Molecule(name='IUPAC methane' formula='CH4' hash='b4057dd')>

To run a optimization on this methane molecule we need to specify the full input as shown below. It should be noted that this function is also organized in such a way where the optimization of many molecules with the same level of theory is most efficient.

In [11]:
options = {
    "keywords": {'coordsys': 'tric'}, # Geometry optimization program options
    "qc_spec": {                      # Quantum chemistry specifications
        "driver": "gradient",
        "method": "HF",
        "basis": "sto-3g",
        "keywords": None,
        "program": "psi4"
    },
}
compute = client.add_procedure("optimization", "geometric", options, [methane])
compute

<ComputeResponse(nsubmitted=1 nexisting=0)>

The `id`s of the submitted optimization can then be queried and examined. As a note the computation is not instantaneous, you may need to wait a moment and requery for this small molecule.

In [15]:
result = client.query_procedures(id=compute.ids)[0]
result

<OptimizationRecord(id='1' status='COMPLETE')>

In [20]:
ch_bond_original = result.get_initial_molecule().measure([0, 1])
ch_bond_optimized = result.get_final_molecule().measure([0, 1])
print(f"Optimized/Original C-H bond {ch_bond_original}/{ch_bond_optimized} (bohr)")

Optimized/Original C-H bond 2.0639574742067777/2.0465935246983378 (bohr)


In [21]:
result.show_history()